### Pre-setup

In [2]:
import os
os.environ['LD_LIBRARY_PATH']='/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/cuda/lib64:/usr/local/cuda/targets/x86_64-linux/lib'
os.environ['CUDA_TOOLKIT_PATH']='/usr/local/cuda'
os.environ['CUDNN_INSTALL_PATH']='/usr/local/cuda'
os.environ['CUDA_HOME']='/usr/local/cuda'
os.environ['NVIDIA_DRIVER_CAPABILITIES']='compute,utility'
os.environ['NVIDIA_VISIBLE_DEVICES']='all'
os.environ['PYTHONPATH']='~/miniconda3/envs/py38/lib/python3.8/site-packages/aimet_common/x86_64-linux-gnu'
os.environ['LD_LIBRARY_PATH'] +=':~/miniconda3/envs/py38/lib/python3.8/site-packages/aimet_common'
os.environ['LD_LIBRARY_PATH'] +=':~/miniconda3/envs/py38/lib/python3.8/site-packages/aimet_common/x86_64-linux-gnu'
os.environ['LD_LIBRARY_PATH'] +=':~/miniconda3/envs/py38/lib/python3.8/site-packages'
os.environ['LD_LIBRARY_PATH'] +=':/usr/lib/x86_64-linux-gnu/'

In [3]:
import sys
sys.path.append('/root/miniconda3/envs/py38/lib/python3.8/site-packages/aimet_common')

### Code

In [4]:
ROOT_DATA_AND_OUTPUTS = './'
N = 16 # number of classes and samples per class
BIWIDTH = 4
BIWIDTH_ACTIVATION = 8 #quantization on the input and output of the layer
DATASET_FOLDER_PATH = '../input/imagenetmini-1000/imagenet-mini/'

# batch size of evaluation is 32
# in the paper they used 2048 images -> 64 batches !!!!!!!!!
#ADAROUND_NUM_BATCHES = 64
ADAROUND_ITERATIONS = 20000
ADAROUND_NUM_BATCHES = 16
#ADAROUND_ITERATIONS = 10000

# in paper their claim to have accuracy ~68.6%

import datetime
DATASET_DIR = f'{ROOT_DATA_AND_OUTPUTS}{DATASET_FOLDER_PATH}'
output_dir = f'{ROOT_DATA_AND_OUTPUTS}output_{datetime.datetime.now().strftime("%Y%m%d_%H%M")}/'
import os
os.makedirs(output_dir, exist_ok=True)

import os
import torch
import Examples.common.image_net_config as image_net_config
from Examples.torch.utils.image_net_evaluator import ImageNetEvaluator
from Examples.torch.utils.image_net_data_loader import ImageNetDataLoader

class ImageNetDataPipeline:

    @staticmethod
    def get_val_dataloader() -> torch.utils.data.DataLoader:
        """
        Instantiates a validation dataloader for ImageNet dataset and returns it
        """
        data_loader = ImageNetDataLoader(DATASET_DIR,
                                         image_size=image_net_config.dataset['image_size'],
                                         batch_size=image_net_config.evaluation['batch_size'],
                                         is_training=False,
                                         num_workers=image_net_config.evaluation['num_workers'],
                                         num_samples_per_class=N).data_loader
        return data_loader

    @staticmethod
    def evaluate(model: torch.nn.Module, use_cuda: bool) -> float:
        """
        Given a torch model, evaluates its Top-1 accuracy on the dataset
        :param model: the model to evaluate
        :param use_cuda: whether or not the GPU should be used.
        """
        evaluator = ImageNetEvaluator(DATASET_DIR, image_size=image_net_config.dataset['image_size'],
                                      batch_size=image_net_config.evaluation['batch_size'],
                                      num_workers=image_net_config.evaluation['num_workers'],
                                      num_val_samples_per_class=N)

        return evaluator.evaluate(model, iterations=None, use_cuda=use_cuda)
    
from torchvision.models import resnet18
model = resnet18(pretrained=True)

# model preperation
from aimet_torch.model_preparer import prepare_model
model = prepare_model(model)

# move to device
use_cuda = False
if torch.cuda.is_available():
    use_cuda = True
    model.to(torch.device('cuda'))
print('Using cuda: {}'.format(use_cuda))

# accuracy of the original model
accuracy = ImageNetDataPipeline.evaluate(model, use_cuda)

from termcolor import colored
print(colored("###########################################################################################################", 'green'))
print(colored(f"Original model accuracy: {accuracy}", 'red'))
print(colored("###########################################################################################################", 'green'))

from aimet_torch.batch_norm_fold import fold_all_batch_norms

_ = fold_all_batch_norms(model, input_shapes=(1, 3, 224, 224))

from aimet_common.defs import QuantScheme
from aimet_torch.quantsim import QuantizationSimModel

dummy_input = torch.rand(1, 3, 224, 224)    # Shape for each ImageNet sample is (3 channels) x (224 height) x (224 width)
if use_cuda:
    dummy_input = dummy_input.cuda()

def pass_calibration_data(sim_model, use_cuda):
    data_loader = ImageNetDataPipeline.get_val_dataloader()
    batch_size = data_loader.batch_size

    if use_cuda:
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')

    sim_model.eval()
    samples = 1000

    batch_cntr = 0
    with torch.no_grad():
        for input_data, target_data in data_loader:

            inputs_batch = input_data.to(device)
            sim_model(inputs_batch)

            batch_cntr += 1
            if (batch_cntr * batch_size) > samples:
                break

sim = QuantizationSimModel(model=model,
                           quant_scheme=QuantScheme.post_training_tf_enhanced,
                           #quant_scheme=QuantScheme.post_training_tf,
                           dummy_input=dummy_input,
                           default_output_bw=BIWIDTH_ACTIVATION,
                           default_param_bw=BIWIDTH,
                           rounding_mode = 'nearest')
                
sim.compute_encodings(forward_pass_callback=pass_calibration_data,
                      forward_pass_callback_args=use_cuda)

accuracy = ImageNetDataPipeline.evaluate(sim.model, use_cuda)
print(colored("###########################################################################################################", 'green'))
print(colored(f"Quantized (nearest) model accuracy: {accuracy}", 'red'))
print(colored("###########################################################################################################", 'green'))

sim = QuantizationSimModel(model=model,
                           quant_scheme=QuantScheme.post_training_tf_enhanced,
                           #quant_scheme=QuantScheme.post_training_tf,
                           dummy_input=dummy_input,
                           default_output_bw=BIWIDTH_ACTIVATION,
                           default_param_bw=BIWIDTH,
                           rounding_mode = 'stochastic')
                
sim.compute_encodings(forward_pass_callback=pass_calibration_data,
                      forward_pass_callback_args=use_cuda)

accuracy = ImageNetDataPipeline.evaluate(sim.model, use_cuda)
print(colored("###########################################################################################################", 'green'))
print(colored(f"Quantized (stochastic) model accuracy: {accuracy}", 'red'))
print(colored("###########################################################################################################", 'green'))

#AdaRound
from aimet_torch.adaround.adaround_weight import Adaround, AdaroundParameters
from aimet_torch.adaround.adaround_weight import AdaroundParameters
#from adaround_weight_new import Adaround

data_loader = ImageNetDataPipeline.get_val_dataloader()
params = AdaroundParameters(data_loader=data_loader, num_batches=ADAROUND_NUM_BATCHES, default_num_iterations=ADAROUND_ITERATIONS)

dummy_input = torch.rand(1, 3, 224, 224)
if use_cuda:
    dummy_input = dummy_input.cuda()

os.makedirs(f'{output_dir}', exist_ok=True)

ada_model = Adaround.apply_adaround(model, dummy_input, params,
                                    path=f'{output_dir}', 
                                    filename_prefix='adaround', 
                                    default_param_bw=BIWIDTH,
                                    default_quant_scheme=QuantScheme.post_training_tf_enhanced
                                    #default_quant_scheme=QuantScheme.post_training_tf
                                   )

# model ready to use
sim = QuantizationSimModel(model=ada_model,
                           dummy_input=dummy_input,
                           quant_scheme=QuantScheme.post_training_tf_enhanced,
                           #quant_scheme=QuantScheme.post_training_tf,
                           default_output_bw=BIWIDTH_ACTIVATION, 
                           default_param_bw=BIWIDTH)

sim.set_and_freeze_param_encodings(encoding_path=f'{output_dir}adaround.encodings')

sim.compute_encodings(forward_pass_callback=pass_calibration_data,
                      forward_pass_callback_args=use_cuda)

accuracy = ImageNetDataPipeline.evaluate(sim.model, use_cuda)
print(colored("###########################################################################################################", 'green'))
print(colored(f"Quantized (AdaRound) model accuracy: {accuracy}", 'red'))
print(colored("###########################################################################################################", 'green'))

dummy_input = dummy_input.cpu()
sim.export(path=output_dir, filename_prefix='resnet18_after_adaround', dummy_input=dummy_input)

2023-06-06 18:26:38,195 - Quant - INFO - Functional         : Adding new module for node: {add} 
2023-06-06 18:26:38,195 - Quant - INFO - Reused/Duplicate   : Adding new module for node: {layer1_0_relu_1} 
2023-06-06 18:26:38,195 - Quant - INFO - Functional         : Adding new module for node: {add_1} 
2023-06-06 18:26:38,196 - Quant - INFO - Reused/Duplicate   : Adding new module for node: {layer1_1_relu_1} 
2023-06-06 18:26:38,196 - Quant - INFO - Functional         : Adding new module for node: {add_2} 
2023-06-06 18:26:38,196 - Quant - INFO - Reused/Duplicate   : Adding new module for node: {layer2_0_relu_1} 
2023-06-06 18:26:38,196 - Quant - INFO - Functional         : Adding new module for node: {add_3} 
2023-06-06 18:26:38,197 - Quant - INFO - Reused/Duplicate   : Adding new module for node: {layer2_1_relu_1} 
2023-06-06 18:26:38,197 - Quant - INFO - Functional         : Adding new module for node: {add_4} 
2023-06-06 18:26:38,197 - Quant - INFO - Reused/Duplicate   : Adding ne

  0% (0 of 123) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--
  2% (3 of 123) |                        | Elapsed Time: 0:00:00 ETA:   0:00:04
  3% (4 of 123) |                        | Elapsed Time: 0:00:00 ETA:   0:00:04
  4% (5 of 123) |                        | Elapsed Time: 0:00:00 ETA:   0:00:06
  5% (7 of 123) |#                       | Elapsed Time: 0:00:00 ETA:   0:00:07
  6% (8 of 123) |#                       | Elapsed Time: 0:00:00 ETA:   0:00:07
  8% (10 of 123) |#                      | Elapsed Time: 0:00:00 ETA:   0:00:07
  8% (11 of 123) |##                     | Elapsed Time: 0:00:00 ETA:   0:00:07
 10% (13 of 123) |##                     | Elapsed Time: 0:00:00 ETA:   0:00:07
 12% (15 of 123) |##                     | Elapsed Time: 0:00:01 ETA:   0:00:08
 13% (16 of 123) |##                     | Elapsed Time: 0:00:01 ETA:   0:00:08
 14% (18 of 123) |###                    | Elapsed Time: 0:00:01 ETA:   0:00:07
 15% (19 of 123) |###                   

2023-06-06 18:26:48,633 - Eval - INFO - Avg accuracy Top 1: 69.755563 Avg accuracy Top 5: 88.600503 on validation Dataset
###########################################################################################################
Original model accuracy: 69.75556269699965
###########################################################################################################
2023-06-06 18:26:48,889 - Utils - INFO - ...... subset to store [Conv_0, BatchNormalization_1]
2023-06-06 18:26:48,890 - Utils - INFO - ...... subset to store [Conv_4, BatchNormalization_5]
2023-06-06 18:26:48,890 - Utils - INFO - ...... subset to store [Conv_7, BatchNormalization_8]
2023-06-06 18:26:48,890 - Utils - INFO - ...... subset to store [Conv_11, BatchNormalization_12]
2023-06-06 18:26:48,890 - Utils - INFO - ...... subset to store [Conv_14, BatchNormalization_15]
2023-06-06 18:26:48,890 - Utils - INFO - ...... subset to store [Conv_18, BatchNormalization_19]
2023-06-06 18:26:48,890 - Utils - INFO - ..

  0% (0 of 123) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--
  1% (2 of 123) |                        | Elapsed Time: 0:00:00 ETA:  00:00:00
  3% (4 of 123) |                        | Elapsed Time: 0:00:00 ETA:   0:00:07
  4% (5 of 123) |                        | Elapsed Time: 0:00:00 ETA:   0:00:07
  5% (7 of 123) |#                       | Elapsed Time: 0:00:00 ETA:   0:00:08
  6% (8 of 123) |#                       | Elapsed Time: 0:00:00 ETA:   0:00:08
  8% (10 of 123) |#                      | Elapsed Time: 0:00:00 ETA:   0:00:08
  8% (11 of 123) |##                     | Elapsed Time: 0:00:00 ETA:   0:00:08
 10% (13 of 123) |##                     | Elapsed Time: 0:00:00 ETA:   0:00:08
 12% (15 of 123) |##                     | Elapsed Time: 0:00:01 ETA:   0:00:08
 13% (16 of 123) |##                     | Elapsed Time: 0:00:01 ETA:   0:00:08
 14% (18 of 123) |###                    | Elapsed Time: 0:00:01 ETA:   0:00:08
 15% (19 of 123) |###                   

2023-06-06 18:27:05,538 - Eval - INFO - Avg accuracy Top 1: 17.863447 Avg accuracy Top 5: 38.470796 on validation Dataset
###########################################################################################################
Quantized (nearest) model accuracy: 17.86344672412407
###########################################################################################################
2023-06-06 18:27:05,673 - Quant - INFO - No config file provided, defaulting to config file at /home/aurif/miniconda3/envs/py38/lib/python3.8/site-packages/aimet_common/quantsim_config/default_config.json
2023-06-06 18:27:05,681 - Quant - INFO - Unsupported op type Squeeze
2023-06-06 18:27:05,682 - Quant - INFO - Unsupported op type Pad
2023-06-06 18:27:05,682 - Quant - INFO - Unsupported op type Mean
2023-06-06 18:27:05,683 - Utils - INFO - ...... subset to store [Conv_0, Relu_2]
2023-06-06 18:27:05,683 - Utils - INFO - ...... subset to store [Conv_4, Relu_6]
2023-06-06 18:27:05,683 - Utils - INFO - 

...

100%|██████████| 68/68 [13:26<00:00, 11.86s/it]

2023-06-06 18:40:48,954 - Quant - INFO - Deleting model inputs from location: /tmp/adaround/
2023-06-06 18:40:48,972 - Quant - INFO - Completed Adarounding Model
2023-06-06 18:40:49,114 - Quant - INFO - No config file provided, defaulting to config file at /home/aurif/miniconda3/envs/py38/lib/python3.8/site-packages/aimet_common/quantsim_config/default_config.json
2023-06-06 18:40:49,123 - Quant - INFO - Unsupported op type Squeeze
2023-06-06 18:40:49,123 - Quant - INFO - Unsupported op type Pad
2023-06-06 18:40:49,123 - Quant - INFO - Unsupported op type Mean
2023-06-06 18:40:49,125 - Utils - INFO - ...... subset to store [Conv_0, Relu_2]
2023-06-06 18:40:49,125 - Utils - INFO - ...... subset to store [Conv_4, Relu_6]
2023-06-06 18:40:49,125 - Utils - INFO - ...... subset to store [Add_9, Relu_10]
2023-06-06 18:40:49,125 - Utils - INFO - ...... subset to store [Conv_11, Relu_13]
2023-06-06 18:40:49,125 - Utils - INFO - ...... subset to store [Add_16, Relu_17]
2023-06-06 18:40:49,125 -

2023-06-06 18:40:49,267 - Dataloader - INFO - Dataset consists of 3923 images in 1000 classes
2023-06-06 18:40:53,798 - Dataloader - INFO - Dataset consists of 3923 images in 1000 classes
2023-06-06 18:40:53,799 - Eval - INFO - No value of iteration is provided, running evaluation on complete dataset.
2023-06-06 18:40:53,799 - Eval - INFO - Evaluating nn.Module for 123 iterations with batch_size 32


  0% (0 of 123) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--
  1% (2 of 123) |                        | Elapsed Time: 0:00:00 ETA:  00:00:00
  3% (4 of 123) |                        | Elapsed Time: 0:00:00 ETA:   0:00:07
  4% (5 of 123) |                        | Elapsed Time: 0:00:00 ETA:   0:00:07
  5% (7 of 123) |#                       | Elapsed Time: 0:00:00 ETA:   0:00:08
  6% (8 of 123) |#                       | Elapsed Time: 0:00:00 ETA:   0:00:08
  8% (10 of 123) |#                      | Elapsed Time: 0:00:00 ETA:   0:00:08
  8% (11 of 123) |##                     | Elapsed Time: 0:00:00 ETA:   0:00:08
 10% (13 of 123) |##                     | Elapsed Time: 0:00:00 ETA:   0:00:08
 12% (15 of 123) |##                     | Elapsed Time: 0:00:01 ETA:   0:00:08
 13% (16 of 123) |##                     | Elapsed Time: 0:00:01 ETA:   0:00:08
 14% (18 of 123) |###                    | Elapsed Time: 0:00:01 ETA:   0:00:08
 15% (19 of 123) |###                   

2023-06-06 18:41:04,054 - Eval - INFO - Avg accuracy Top 1: 67.672229 Avg accuracy Top 5: 87.592266 on validation Dataset
###########################################################################################################
Quantized (AdaRound) model accuracy: 67.67222936366632
###########################################################################################################
2023-06-06 18:41:04,497 - Utils - INFO - successfully created onnx model with 48/49 node names updated
2023-06-06 18:41:04,625 - Quant - INFO - Layers excluded from quantization: []
